In [ ]:
import tensorflow as tf

print(tf.config.list_physical_devices()) 

In [ ]:
import os
import re
import shutil

# Define source directory (where class folders are located)
source_dir = 'dataset/UCF_Crime/Train'  # Adjust path as needed
target_dir = 'dataset/UCF_Crime_Organized/train'  # Adjust target path as needed

# Function to organize frames by video and frame numbers for each class
def organize_all_classes(source_dir, target_dir):
    # Iterate over each class folder in the source directory
    for class_name in range(0,1):
        class_path = os.path.join(source_dir, 'NormalVideos')
        
        if os.path.isdir(class_path):
            # Create a directory for each class in the target path
            class_target_dir = os.path.join(target_dir, 'NormalVideos')
            os.makedirs(class_target_dir, exist_ok=True)
            
            # Regex pattern to match filenames (e.g., Abuse001_x264_0.png)
            pattern = rf'Normal_Videos(\d+)_x264_(\d+)\.png'
            
            # Iterate over each file in the class folder
            for filename in os.listdir(class_path):
                match = re.match(pattern, filename)
                if match:
                    video_number = match.group(1)  # Extract video number
                    frame_number = match.group(2)  # Extract frame number

                    # Create a directory for each video within the class folder
                    video_dir = os.path.join(class_target_dir, f"video_{int(video_number)}")
                    os.makedirs(video_dir, exist_ok=True)

                    # Move each frame to the corresponding video directory
                    src_path = os.path.join(class_path, filename)
                    dest_path = os.path.join(video_dir, f"frame_{int(frame_number):04d}.png")
                    shutil.copy(src_path, dest_path)

                    print(f"Moved {filename} to {dest_path}")

# Run the function
organize_all_classes(source_dir, target_dir)


In [ ]:
import json

# Load annotations
with open('/Users/vishnu/garage/cs512-f24-sellamshanmugavel-vishnupriyan/project/src/dataset/annotations/train.json', 'r') as f:
    annotations = json.load(f)

# Print example of parsed annotations
print(annotations["Abuse/Abuse001_x264.mp4"])

# Initialize variables to store the maximum and minimum interval durations
max_interval = 0
min_interval = float('inf')

# Iterate through each video's annotations
for video, segments in annotations.items():
    for segment in segments:
        interval = segment['end'] - segment['start']
        if interval > max_interval:
            max_interval = interval
        if interval < min_interval:
            min_interval = interval

# If there were no segments, set min_interval to 0
if min_interval == float('inf'):
    min_interval = 0

print("Maximum interval duration:", max_interval)
print("Minimum interval duration:", min_interval)

In [ ]:
# Initialize counters and lists
below_16_frames_count = 0
above_100_seconds = []
negative_intervals = []

# Iterate through each video's annotations
for video, segments in annotations.items():
    for segment in segments:
        interval = segment['end'] - segment['start']

        # Check for intervals with fewer than 16 frames
        if interval < 16:
            below_16_frames_count += 1

        # Check for intervals greater than 100 seconds
        if interval > 100:
            above_100_seconds.append((video, segment))

        # Check for negative intervals
        if interval < 0:
            negative_intervals.append((video, segment))

# Output results
print("Number of intervals below 16 frames:", below_16_frames_count)

print("\nIntervals with durations above 100 seconds:", len(above_100_seconds))
for video, segment in above_100_seconds:
    print(f"{video} - start: {segment['start']}, end: {segment['end']}")

print("\nNegative intervals:",len(negative_intervals))
for video, segment in negative_intervals:
    print(f"{video} - start: {segment['start']}, end: {segment['end']}")